<a href="https://colab.research.google.com/github/heddaenger/FakeNews/blob/main/Logreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import Callable # Extra: read about type hints: https://docs.python.org/3/library/typing.html
import warnings



# needs to run before importing pandas. Silences a harmless pandas FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import spacy
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
import string
from scipy.sparse import csr_matrix
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, f1_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import numpy as np

In [ ]:
#Load data
class CSVLoader:
    """ Load a CSV file into a pandas DataFrame."""

    @staticmethod
    def fetch(path : str, **kwargs) -> pd.DataFrame:
        return pd.read_csv(path, **kwargs)

    def load(self, path : str, prep_func : Callable | None, **kwargs) -> pd.DataFrame:
        data = self.fetch(path, **kwargs)
        if isinstance(prep_func, Callable):
            return prep_func(data)
        return data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def preprocess_label(df: pd.DataFrame) -> pd.DataFrame:
    """ A data preparation function for the fake news dataset."""
    return (
        df
        .drop(columns=['Unnamed: 0'], errors='ignore')
        .astype({"text": 'string', "label": 'category'})
    )

dataf = CSVLoader().load(
    path='/content/drive/MyDrive/train.csv',
    prep_func=preprocess_label
)

In [ ]:
def preprocess_label(df: pd.DataFrame) -> pd.DataFrame:
    """ A data preparation function for the fake news dataset."""
    return (
        df
        .drop(columns=['Unnamed: 0'], errors='ignore')
        .astype({"text": 'string'})
    )

test_raw = CSVLoader().load(
    path='/content/drive/MyDrive/test.csv',
    prep_func=preprocess_label
)

In [ ]:
df = dataf.dropna()

In [ ]:
# Drop rows with missing values and reset the index
df_unprocessed = dataf.dropna().reset_index(drop=True)

In [ ]:
df_unprocessed =df_unprocessed.head(2000)

In [ ]:
df_unprocessed.shape

(2000, 5)

In [ ]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
testdf = test_raw.dropna()

In [ ]:
test_unprocessed = testdf.dropna().reset_index(drop=True)

In [ ]:
test_unprocessed = test_unprocessed.head(2000)

In [ ]:
test_unprocessed.shape

(2000, 4)

In [ ]:
test_unprocessed.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [ ]:
test_data = test_unprocessed.drop(['author', 'title', 'id'], axis=1)

In [ ]:
test_data.head()

,text
0,"PALO ALTO, Calif. — After years of scorning..."
1,Videos #NoDAPL: Native American Leaders Vow to...
2,"If at first you don’t succeed, try a different..."
3,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
4,"Sunday on NBC’s “Meet the Press,” House Minori..."


In [ ]:
#Making the train/test split with unprocessed data (unvectorized and untokenized)
(
    X_train,
    X_test,
    y_train,
    y_test
) = train_test_split(
    df_unprocessed['text'],
    df_unprocessed['label'],
    test_size=0.4, # toggle as needed
    stratify=df_unprocessed["label"],
    random_state=42
  )

In [ ]:
#Checking the distribution of real and fake news
y_train.value_counts()

label
0    677
1    523
Name: count, dtype: int64

In [ ]:
#Making the pipeline for logistic regression.
logistic_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('logreg', LogisticRegression()),
])

# Defining the hyperparameters for the model to search through.
parameters = {
    'tfidf__max_df': (0.5, 0.75, 1.0),
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or bigrams
    'logreg__C': np.logspace(-4, 4, 20),
    'logreg__penalty': ['l1', 'l2'],
    'logreg__solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Running the randomized search.
random_search_logistic = RandomizedSearchCV(logistic_pipeline, parameters, n_iter=5, cv=5, verbose=1, random_state=42)

# Fitting the model.
random_search_logistic.fit(X_train, y_train)

# Getting the best hyperparameters.
best_parameters = random_search_logistic.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(f"{param_name}: {best_parameters[param_name]}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

logreg__C: 3792.690190732246
logreg__penalty: l1
logreg__solver: liblinear
tfidf__max_df: 0.75
tfidf__ngram_range: (1, 2)


In [ ]:
#Making the new pipeline with the best parameters and fit the model to those.
log_reg_tuned_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_df=best_parameters['tfidf__max_df'],
        ngram_range=best_parameters['tfidf__ngram_range']
    )),
    ('logreg', LogisticRegression(
        C=best_parameters['logreg__C'],
        penalty=best_parameters['logreg__penalty'],
        solver=best_parameters['logreg__solver']
    )),
])

log_reg_tuned_pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=(1, 2))),
                ('logreg',
                 LogisticRegression(C=3792.690190732246, penalty='l1',
                                    solver='liblinear'))])

In [ ]:
# Predicting on the test set.
y_pred_log_test = log_reg_tuned_pipeline.predict(X_test)
y_pred_log_train = log_reg_tuned_pipeline.predict (X_train)

# Evaluating the model and printing accuracy on train/test to check for overfitting.
print(classification_report(y_test, y_pred_log_test, target_names=['real', 'fake']))
print("Accuracy train:", accuracy_score(y_train, y_pred_log_train))
print("Accuracy test:", accuracy_score(y_test, y_pred_log_test))

              precision    recall  f1-score   support

        real       0.91      0.96      0.93       452
        fake       0.94      0.87      0.90       348

    accuracy                           0.92       800
   macro avg       0.92      0.91      0.92       800
weighted avg       0.92      0.92      0.92       800

Accuracy train: 1.0
Accuracy test: 0.92


In [ ]:
print(test_data.shape)

(2000, 1)


In [ ]:
# Predicting on the test set.
pred_log_final = log_reg_tuned_pipeline.predict(test_data)

In [ ]:
results = pd.DataFrame({
    'Predictions': pred_log_final
})

In [ ]:
# Getting probability estimates for the predictions
probabilities = log_reg_tuned_pipeline.predict_proba(test_data)[:, 1]  # Assuming class '1' probabilities

# Creating a DataFrame to hold the test data identifiers, predictions, and probabilities
results = pd.DataFrame({
    'Predictions': pred_log_final,
    'Probability': probabilities
})

# Save to CSV
results.to_csv('enhanced_predictions.csv', index=False)

In [ ]:
print(test_data.shape)

(2000, 1)


In [ ]:
# Save the results to a new CSV file
results.to_csv('predictions.csv', index=False)